In [7]:
import torch

pt_file = "/media/george-vengrovski/Desk SSD/BirdJEPA/llb3_val/llb3_0001_2018_04_23_14_18_03.pt"
data = torch.load(pt_file)
print(data['s'].shape)
print(data['labels'].shape)

torch.Size([128, 73])
torch.Size([73])
